In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb

from keras.models import Sequential
from keras.layers.core import Dense, Activation, Merge, Reshape, Dropout
from keras.layers.embeddings import Embedding
from keras.optimizers import SGD
from keras.layers.normalization import BatchNormalization

from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from collections import defaultdict

%matplotlib inline
pd.set_option('display.max_columns', None)

Using TensorFlow backend.


In [2]:
train = pd.read_csv('../data/processed/train-normalized.csv.gz', skipinitialspace = True)

In [3]:
test = pd.read_csv('../data/raw/test.csv.zip', skipinitialspace = True)

/Users/petr/Applications/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
train.head()

,fecha_dato,ncodpers,sexo,age,ind_nuevo,antiguedad,indrel,indresi,indext,ind_actividad_cliente,renta,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1,ind_empleado_A,ind_empleado_B,ind_empleado_F,ind_empleado_N,ind_empleado_S,indrel_1mes_1.0,indrel_1mes_2.0,indrel_1mes_3.0,indrel_1mes_4.0,indrel_1mes_5.0,indrel_1mes_6.0,tiprel_1mes_A,tiprel_1mes_I,tiprel_1mes_N,tiprel_1mes_P,tiprel_1mes_R,segmento_01 - TOP,segmento_02 - PARTICULARES,segmento_03 - UNIVERSITARIO
0,2015-05-28,15889,1.0,0.341463,0.0,0.999989,1.0,1.0,0.0,1.0,0.011287,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,2015-06-28,15889,1.0,0.341463,0.0,0.999989,1.0,1.0,0.0,1.0,0.011287,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,2016-05-28,15889,1.0,0.341463,0.0,0.999999,1.0,1.0,0.0,1.0,0.011287,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,2015-05-28,15890,1.0,0.378049,0.0,0.999990,1.0,1.0,0.0,1.0,0.002473,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,2015-06-28,15890,1.0,0.378049,0.0,0.999990,1.0,1.0,0.0,1.0,0.002473,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [5]:
features = ['ncodpers', 'sexo', 'age', 'ind_nuevo', 'antiguedad', 
            'indrel', 'indresi', 'indext', 'ind_actividad_cliente', 'renta', 
            'ind_empleado_A', 'ind_empleado_B', 'ind_empleado_F', 'ind_empleado_N', 
            'ind_empleado_S', 'indrel_1mes_1.0', 'indrel_1mes_2.0', 'indrel_1mes_3.0', 
            'indrel_1mes_4.0', 'indrel_1mes_5.0', 'indrel_1mes_6.0', 'tiprel_1mes_A', 
            'tiprel_1mes_I', 'tiprel_1mes_N', 'tiprel_1mes_P', 'tiprel_1mes_R']

products = ['ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1',
            'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1',
            'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1',
            'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1',
            'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1',
            'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1',
            'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1',
            'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1']

In [6]:
def process_data(df, cust_dict):
    x_vars_list = []
    y_vars_list = []

    for idx, row in df.iterrows():

        cust_id = int(row['ncodpers'])

        if row['fecha_dato'] in ['2015-05-28', '2016-05-28']:   
            target_list = row[products].values
            cust_dict[cust_id] =  target_list[:]
            continue

        x_vars = row[features].values

        if row['fecha_dato'] == '2016-06-28':
            prev_target_list = cust_dict.get(cust_id, [0]*len(products))
            x_vars_list.append(np.concatenate((x_vars, prev_target_list)))
        elif row['fecha_dato'] == '2015-06-28': # Basically it saves only those clients who bought something new in a last year for same month.
            prev_target_list = cust_dict.get(cust_id, [0]*len(products))
            target_list = row[products].values
            new_products = [max(x1 - x2,0) for (x1, x2) in zip(target_list, prev_target_list)]
            if sum(new_products) > 0:
                for ind, prod in enumerate(new_products):
                    if prod>0:
                        assert len(prev_target_list) == len(products)
                        x_vars_list.append(np.concatenate((x_vars, prev_target_list)))
                        y_vars_list.append(ind)
    
    return x_vars_list, y_vars_list, cust_dict

In [7]:
X_train, y_train, cust_dict = process_data(train, {})

In [8]:
X_test, y_test, cust_dict = process_data(test, cust_dict)

In [13]:
def runXGB(train_X, train_y, seed_val=123):
    param = {}
    param['objective'] = 'multi:softprob'
    param['eta'] = 0.05
    param['max_depth'] = 4
    param['silent'] = 1
    param['num_class'] = 24
    param['eval_metric'] = "mlogloss"
    param['min_child_weight'] = 2
    param['subsample'] = 0.9
    param['colsample_bytree'] = 0.9
    param['seed'] = seed_val
    num_rounds = 190

    plst = list(param.items())
    xgtrain = xgb.DMatrix(train_X, label=train_y)
    model = xgb.train(plst, xgtrain, num_rounds)    
    return model

In [14]:
print("Building model..")
model = runXGB(np.array(X_train), np.array(y_train), seed_val=0)
del X_train, y_train
print("Predicting..")
xgtest = xgb.DMatrix(np.array(X_test))
preds = model.predict(xgtest)
del X_test, xgtest

print("Getting the top products..")
target_cols = np.array(products)
preds = np.argsort(preds, axis=1)
preds = np.fliplr(preds)[:,:8]
test_id = np.array(test.ncodpers)

final_preds = [" ".join(list(products[pred])) for pred in preds]
out_df = pd.DataFrame({'ncodpers':test_id, 'added_products':final_preds})
out_df.to_csv('sub_xgb_new.csv', index=False)
print(datetime.datetime.now()-start_time)

Building model..
Predicting..


ValueError: could not convert string to float: 'N'

In [ ]:
ids = data['ncodpers']
data.drop(['ncodpers'], axis=1, inplace=True)

In [ ]:
models = {}
model_preds = {}
id_preds = defaultdict(list)
products = ['ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1',
            'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1',
            'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1',
            'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1',
            'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1',
            'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1',
            'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1',
            'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1']

In [ ]:
for product in products:
        print(product)
        y_train = data[product]
        x_train = data.drop(product, 1)
        
        clf = LogisticRegression()
        clf.fit(x_train, y_train)
        p_train = clf.predict_proba(x_train)[:,1]
        
        models[product] = clf
        model_preds[product] = p_train
        for id, p in zip(ids, p_train):
            id_preds[id].append(p)
            
        print(roc_auc_score(y_train, p_train))

In [ ]:
already_active = defaultdict(list)

for i in range(len(data)):  
    id_ = ids.at[i]
    active_products = []
    
    for p in products:
        if data.at[i, p] == 1.0:
            active_products.append(p)
    
    already_active[id_] = active_products

In [ ]:
most_popular = ['ind_cco_fin_ult1', 'ind_recibo_ult1', 'ind_ctop_fin_ult1', 'ind_cno_fin_ult1', 'ind_ecue_fin_ult1', 'ind_nom_pens_ult1', 'ind_nomina_ult1']

train_preds = {}

for id_ in test.ncodpers.values:
    preds_probas = id_preds[id_]
    active = already_active[id_]    
    predicted = []
    
    for prod_proba_idx in np.argsort(preds_probas)[::-1]:
        if products[prod_proba_idx] not in active and len(predicted) < 7:
            predicted.append(products[prod_proba_idx])
    
    train_preds[id_] = most_popular if len(predicted) == 0 else predicted

In [ ]:
for ncodper in test.ncodpers.values:        
    train_preds[ncodper] = ' '.join(train_preds[ncodper])

In [ ]:
predictions = pd.DataFrame(list(train_preds.items()), columns=['ncodpers', 'added_products'])
predictions.head()

In [ ]:
predictions.to_csv('../data/predicted.csv.gz', index=False, compression='gzip')